##### Copyright 2023 The TF-Agents Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Políticas

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/agents/tutorials/3_policies_tutorial"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/agents/tutorials/3_policies_tutorial.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/agents/tutorials/3_policies_tutorial.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/agents/tutorials/3_policies_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a></td>
</table>

## Introducción

En la terminología del Aprendizaje por Refuerzo, las políticas se encargan de asignar una observación del entorno a una acción o a una distribución de acciones. En TF-Agents, las observaciones del entorno están contenidas en una tupla nombrada `TimeStep('step_type', 'discount', 'reward', 'observation')`, y las políticas asignan pasos de tiempo a acciones o distribuciones de acciones. La mayoría de las políticas usan `timestep.observation`, algunas políticas usan `timestep.step_type` (por ejemplo, para restablecer el estado al principio de un episodio en políticas de estado), pero `timestep.discount` y `timestep.reward` se suelen ignorar.

Las políticas están relacionadas con otros componentes de TF-Agents de la siguiente manera. La mayoría de las políticas tienen una red neuronal para calcular acciones o distribuciones de acciones a partir de TimeSteps. Los agentes pueden contener una o más políticas para diferentes propósitos, por ejemplo, una política principal que se entrena para la implementación, y una política ruidosa para la recopilación de datos. Las políticas pueden guardarse o restaurarse, y se pueden utilizar independientemente del agente para la recopilación de datos, la evaluación, etc.

Algunas políticas son más fáciles de escribir en Tensorflow (por ejemplo, las que tienen una red neuronal), mientras que otras son más fáciles de escribir en Python (por ejemplo, las que siguen un guión de acciones). Así que en TF Agents permitimos tanto políticas en Python como en Tensorflow. Además, es posible que las políticas escritas en TensorFlow deban utilizarse en un entorno Python, o viceversa, por ejemplo, una política TensorFlow se utiliza para el entrenamiento, pero más tarde se implementa en un entorno Python de producción. Para que esto sea más fácil, proporcionamos envoltorios para la conversión entre las políticas de Python y TensorFlow.

Otra clase de políticas muy interesante son los envoltorios de políticas, que modifican una política determinada de una manera determinada, por ejemplo, para añadir un tipo concreto de ruido, convertir una política estocástica en una versión greedy o epsilon-greedy, mezclar aleatoriamente varias políticas, etc. 

## Preparación

Si todavía no ha instalado tf-agents, ejecute los siguientes comandos:

In [ ]:
!pip install tf-agents

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np

from tf_agents.specs import array_spec
from tf_agents.specs import tensor_spec
from tf_agents.networks import network

from tf_agents.policies import py_policy
from tf_agents.policies import random_py_policy
from tf_agents.policies import scripted_py_policy

from tf_agents.policies import tf_policy
from tf_agents.policies import random_tf_policy
from tf_agents.policies import actor_policy
from tf_agents.policies import q_policy
from tf_agents.policies import greedy_policy

from tf_agents.trajectories import time_step as ts

## Políticas de Python

La interfaz para políticas de Python se define en `policies/py_policy.PyPolicy`. Estos son los principales métodos:


In [ ]:
class Base(object):

  @abc.abstractmethod
  def __init__(self, time_step_spec, action_spec, policy_state_spec=()):
    self._time_step_spec = time_step_spec
    self._action_spec = action_spec
    self._policy_state_spec = policy_state_spec

  @abc.abstractmethod
  def reset(self, policy_state=()):
    # return initial_policy_state.
    pass

  @abc.abstractmethod
  def action(self, time_step, policy_state=()):
    # return a PolicyStep(action, state, info) named tuple.
    pass

  @abc.abstractmethod
  def distribution(self, time_step, policy_state=()):
    # Not implemented in python, only for TF policies.
    pass

  @abc.abstractmethod
  def update(self, policy):
    # update self to be similar to the input `policy`.
    pass

  @property
  def time_step_spec(self):
    return self._time_step_spec

  @property
  def action_spec(self):
    return self._action_spec

  @property
  def policy_state_spec(self):
    return self._policy_state_spec

El método más importante es `action(time_step)` que asigna un `time_step` que contiene una observación del entorno a una tupla nombrada PolicyStep que contiene los siguientes atributos:

- `action`: la acción que se aplicará al entorno.
- `state`: el estado de la política (por ejemplo, estado de RNN) que se ingresará a la siguiente llamada a la acción.
- `info`: información complementaria opcional, como las probabilidades logarítmicas de las acciones.

`time_step_spec` y `action_spec` son especificaciones para el paso de tiempo de entrada y la acción de salida. Las políticas también tienen una función `reset` que generalmente se usa para restablecer el estado de las políticas de estado. La función `update(new_policy)` actualiza `self` hacia `new_policy`.

Veamos ahora algunos ejemplos de políticas de Python.


### Ejemplo 1: política aleatoria de Python

Un ejemplo sencillo de una `PyPolicy` es `RandomPyPolicy` que genera acciones aleatorias para la action_spec discreta o continua determinada. Se ignora la entrada `time_step`.

In [ ]:
action_spec = array_spec.BoundedArraySpec((2,), np.int32, -10, 10)
my_random_py_policy = random_py_policy.RandomPyPolicy(time_step_spec=None,
    action_spec=action_spec)
time_step = None
action_step = my_random_py_policy.action(time_step)
print(action_step)
action_step = my_random_py_policy.action(time_step)
print(action_step)

### Ejemplo 2: política secuenciada de Python

Una política secuenciada reproduce una secuencia de comandos de acciones que se representa como una lista de tuplas `(num_repeats, action)`. Cada vez que se llama la función `action`, devuelve la siguiente acción de la lista hasta alcanzar la cantidad de repeticiones que se especificó, y luego continúa con la siguiente acción de la lista. Se puede llamar al método `reset` para que empiece a ejecutarse desde el principio de la lista.

In [ ]:
action_spec = array_spec.BoundedArraySpec((2,), np.int32, -10, 10)
action_script = [(1, np.array([5, 2], dtype=np.int32)), 
                 (0, np.array([0, 0], dtype=np.int32)), # Setting `num_repeats` to 0 will skip this action.
                 (2, np.array([1, 2], dtype=np.int32)), 
                 (1, np.array([3, 4], dtype=np.int32))]

my_scripted_py_policy = scripted_py_policy.ScriptedPyPolicy(
    time_step_spec=None, action_spec=action_spec, action_script=action_script)

policy_state = my_scripted_py_policy.get_initial_state()
time_step = None
print('Executing scripted policy...')
action_step = my_scripted_py_policy.action(time_step, policy_state)
print(action_step)
action_step= my_scripted_py_policy.action(time_step, action_step.state)
print(action_step)
action_step = my_scripted_py_policy.action(time_step, action_step.state)
print(action_step)

print('Resetting my_scripted_py_policy...')
policy_state = my_scripted_py_policy.get_initial_state()
action_step = my_scripted_py_policy.action(time_step, policy_state)
print(action_step)

## Políticas de TensorFlow

Las políticas de TensorFlow siguen la misma interfaz que las políticas de Python. Veamos algunos ejemplos.

### Ejemplo 1: política aleatoria de TF

Se puede usar una RandomTFPolicy para generar acciones aleatorias de acuerdo con una `action_spec` discreta o continua dada. Se ignora la entrada `time_step`.


In [ ]:
action_spec = tensor_spec.BoundedTensorSpec(
    (2,), tf.float32, minimum=-1, maximum=3)
input_tensor_spec = tensor_spec.TensorSpec((2,), tf.float32)
time_step_spec = ts.time_step_spec(input_tensor_spec)

my_random_tf_policy = random_tf_policy.RandomTFPolicy(
    action_spec=action_spec, time_step_spec=time_step_spec)
observation = tf.ones(time_step_spec.observation.shape)
time_step = ts.restart(observation)
action_step = my_random_tf_policy.action(time_step)

print('Action:')
print(action_step.action)

### Ejemplo 2: Política de actor

Una política de actor se puede crear mediante el uso de una red que asigne `time_steps` a las acciones o de una red que asigne `time_steps` a distribuciones de acciones.


#### Cómo usar una red de acción

Una red se puede definir de la siguiente manera:

In [ ]:
class ActionNet(network.Network):

  def __init__(self, input_tensor_spec, output_tensor_spec):
    super(ActionNet, self).__init__(
        input_tensor_spec=input_tensor_spec,
        state_spec=(),
        name='ActionNet')
    self._output_tensor_spec = output_tensor_spec
    self._sub_layers = [
        tf.keras.layers.Dense(
            action_spec.shape.num_elements(), activation=tf.nn.tanh),
    ]

  def call(self, observations, step_type, network_state):
    del step_type

    output = tf.cast(observations, dtype=tf.float32)
    for layer in self._sub_layers:
      output = layer(output)
    actions = tf.reshape(output, [-1] + self._output_tensor_spec.shape.as_list())

    # Scale and shift actions to the correct range if necessary.
    return actions, network_state

En TensorFlow la mayoría de las capas de red se diseñaron para operaciones por lotes, por lo que esperamos que los time_steps de entrada estén agrupados por lotes, y que la salida de la red también lo esté. Además, la red es responsable de producir acciones en el intervalo correcto de la action_spec dada. De forma convencional, esto se hace usando, por ejemplo, una activación tanh para que la capa final produzca acciones en [-1, 1] y luego escalando y desplazando esto al intervalo correcto como la action_spec de entrada (por ejemplo, consulte `tf_agents/agents/ddpg/networks.actor_network()`).

Ahora, podemos crear una política de actor utilizando la red que mencionamos anteriormente.

In [ ]:
input_tensor_spec = tensor_spec.TensorSpec((4,), tf.float32)
time_step_spec = ts.time_step_spec(input_tensor_spec)
action_spec = tensor_spec.BoundedTensorSpec((3,),
                                            tf.float32,
                                            minimum=-1,
                                            maximum=1)

action_net = ActionNet(input_tensor_spec, action_spec)

my_actor_policy = actor_policy.ActorPolicy(
    time_step_spec=time_step_spec,
    action_spec=action_spec,
    actor_network=action_net)

Podemos aplicarlo a cualquier lote de time_steps que sigan a time_step_spec:

In [ ]:
batch_size = 2
observations = tf.ones([2] + time_step_spec.observation.shape.as_list())

time_step = ts.restart(observations, batch_size)

action_step = my_actor_policy.action(time_step)
print('Action:')
print(action_step.action)

distribution_step = my_actor_policy.distribution(time_step)
print('Action distribution:')
print(distribution_step.action)

En el ejemplo anterior, creamos la política a partir de una red de acciones que produce un tensor de acciones. En este caso, `policy.distribution(time_step)` es una distribución determinista (delta) en torno a la salida de `policy.action(time_step)`. Una forma de producir una política estocástica consiste en envolver la política de actor en un envoltorio de política que agregue ruido a las acciones. Otra forma es crear la política de actor mediante el uso de una red de distribución de acciones en lugar de una red de acciones, como se muestra a continuación.

#### Cómo usar una red de distribución de acciones

In [ ]:
class ActionDistributionNet(ActionNet):

  def call(self, observations, step_type, network_state):
    action_means, network_state = super(ActionDistributionNet, self).call(
        observations, step_type, network_state)

    action_std = tf.ones_like(action_means)
    return tfp.distributions.MultivariateNormalDiag(action_means, action_std), network_state


action_distribution_net = ActionDistributionNet(input_tensor_spec, action_spec)

my_actor_policy = actor_policy.ActorPolicy(
    time_step_spec=time_step_spec,
    action_spec=action_spec,
    actor_network=action_distribution_net)

action_step = my_actor_policy.action(time_step)
print('Action:')
print(action_step.action)
distribution_step = my_actor_policy.distribution(time_step)
print('Action distribution:')
print(distribution_step.action)

Tenga en cuenta que, en el ejemplo anterior, las acciones se ajustan al intervalo de la especificación de acción dada [-1, 1]. Esto se debe a que un argumento constructor de ActorPolicy clip=True por defecto. Si se establece en false, se devolverán las acciones producidas por la red sin recortar. 

Las políticas estocásticas pueden convertirse en políticas deterministas si, por ejemplo, se utiliza un envoltorio GreedyPolicy que elige `stochastic_policy.distribution().mode()` como acción, y una distribución determinista o delta en torno a esta acción codiciosa como `distribution()`.

### Ejemplo 3: política Q

Una política Q se usa en agentes como DQN y está basada en una red Q que predice un valor Q para cada acción discreta. Para un paso de tiempo dado, la distribución de las acciones en la política Q es una distribución categórica que usa los valores q como logits.


In [ ]:
input_tensor_spec = tensor_spec.TensorSpec((4,), tf.float32)
time_step_spec = ts.time_step_spec(input_tensor_spec)
action_spec = tensor_spec.BoundedTensorSpec((),
                                            tf.int32,
                                            minimum=0,
                                            maximum=2)
num_actions = action_spec.maximum - action_spec.minimum + 1


class QNetwork(network.Network):

  def __init__(self, input_tensor_spec, action_spec, num_actions=num_actions, name=None):
    super(QNetwork, self).__init__(
        input_tensor_spec=input_tensor_spec,
        state_spec=(),
        name=name)
    self._sub_layers = [
        tf.keras.layers.Dense(num_actions),
    ]

  def call(self, inputs, step_type=None, network_state=()):
    del step_type
    inputs = tf.cast(inputs, tf.float32)
    for layer in self._sub_layers:
      inputs = layer(inputs)
    return inputs, network_state


batch_size = 2
observation = tf.ones([batch_size] + time_step_spec.observation.shape.as_list())
time_steps = ts.restart(observation, batch_size=batch_size)

my_q_network = QNetwork(
    input_tensor_spec=input_tensor_spec,
    action_spec=action_spec)
my_q_policy = q_policy.QPolicy(
    time_step_spec, action_spec, q_network=my_q_network)
action_step = my_q_policy.action(time_steps)
distribution_step = my_q_policy.distribution(time_steps)

print('Action:')
print(action_step.action)

print('Action distribution:')
print(distribution_step.action)

## Envoltorios de políticas

Los envoltorios de políticas se pueden utilizar para envolver y modificar una política dada, por ejemplo, añadir ruido. Los envoltorios de políticas son una subclase de Política (Python/TensorFlow) y por lo tanto se pueden utilizar como cualquier otra política. 

### Ejemplo: política codiciosa

Se puede utilizar un envoltorio codicioso para envolver cualquier política de TensorFlow que implemente `distribution()`. `GreedyPolicy.action()` devolverá `wrapped_policy.distribution().mode()` y `GreedyPolicy.distribution()` es una distribución determinista/delta en torno a `GreedyPolicy.action()`:

In [ ]:
my_greedy_policy = greedy_policy.GreedyPolicy(my_q_policy)

action_step = my_greedy_policy.action(time_steps)
print('Action:')
print(action_step.action)

distribution_step = my_greedy_policy.distribution(time_steps)
print('Action distribution:')
print(distribution_step.action)